In [4]:
import pandas as pd
import numpy as np
import sqlite3
import dash
import dash_table
from dash import dcc, html
import plotly.express as px

# Conecta ao banco de dados SQLite
conn = sqlite3.connect('C:/Users/santo/Documents/GitHubPublished/DataScienceProject/database/ecommerceProject.db')

# Dicionário de queries SQL
dictQuery = {
   
    "querytest": '''
    WITH MonthlyRevenue AS (
    SELECT
    STRFTIME('%m-%Y', order_date) AS month,
    ROUND(SUM(order_value),2) AS total_revenue,
    oi.order_id,
    oi.product_id,
    ROW_NUMBER() OVER (PARTITION BY STRFTIME('%m-%Y', order_date) ORDER BY SUM(order_value) DESC) AS rank
    FROM orders o
    INNER JOIN
    order_items oi
    ON o.order_id = oi.order_id
    GROUP BY
    month,
    o.order_id,
    oi.product_id
)
SELECT
    mr.month,
    c.category_name,
    mr.total_revenue
FROM
    MonthlyRevenue mr
INNER JOIN
    products p
    ON p.product_id = mr.product_id
INNER JOIN
    categories c
    ON c.category_id = p.category_id
WHERE mr.rank = 1
--and month like '01-%'
and month like '%-2022'
''',
}

def runQuery(queryKey, params=()):
    """Executa uma consulta SQL e retorna o resultado como um DataFrame."""
    query = dictQuery[queryKey]
    return pd.read_sql(query, conn, params=params)

# Executa as consultas para obter os DataFrames

tQueryTest = runQuery("querytest")

# Inicia o aplicativo Dash
app = dash.Dash(__name__)

# Define o layout com tabelas interativas
app.layout = html.Div([
    html.H1("Tabelas do Banco de Dados Ecommerce", style={"textAlign": "center"}),
    
    html.H2("test"),
    dash_table.DataTable(
        id='customers-table',
        columns=[{"name": col, "id": col} for col in tQueryTest.columns],
        data=tQueryTest.to_dict('records'),
        style_table={'height': '400px', 'overflowY': 'auto'}
    ),

])

if __name__ == '__main__':
    app.run(debug=True)  # Corrigido para 'app.run' em vez de 'app.run_server'
